<center><h1>正则表达式</h1></center>

# 正则表达式

    正则表达式描述了一种字符串匹配的模式，可以用来检查一个字符串是否含有某种子串、将匹配的子串替换或者从某个字符串中取出符合某个条件的子串等。


* 基本规则 
* match()、compile() 、findall()、rearch() 
* 字符串的替换和修改 
* 格式化输出 


# 5.1 正则表达式基础

例子：在爬取某网页中的所有的图片时，需要进行匹配，因为图片有jpg、png、gif等格式。下面的代码对百度贴吧网站上的jpg图片进行了匹配和下载。


【例5-1】对网站上的图片进行匹配并下载。


In [2]:
import re                 # 导入正则表达式
import urllib.request     # 获取网页源代码 
# 获取网页源代码
def getHtml(url):
   page = urllib.request.urlopen(url)   # 打开url，返回页面对象
   html = page.read().decode('utf-8')   # 读取页面源代码
   return html
# 获得图片地址
def getImg(html):
   reg = r'src="(.*?\.jpg)" size="' #定义一个正则表达式匹配页面当中的图片
   imgre = re.compile(reg)           #为了让正则表达式更快，给它来个编译
   imglist = re.findall(imgre, html) #通过正则表达式返回所有数据列表
   # 根据地址逐个进行下载
   x = 0 
   for imgurl in imglist:
       urllib.request.urlretrieve(imgurl,'image/%s.jpg' % x)
html = getHtml("https://tieba.baidu.com/p/5154221980")
getImg(html)
# urlretrieve直接将远程数据下载到本地


我们在网上填表时，经常需要填写手机号码，当只有输入数字才被接收时，就可以用正则表达式去匹配数字。一个数字可以用“\d”匹配，而一个字母或数字可以用“\w”匹配，“.”可以匹配任意字符。  
> '00\d'可以匹配'007'，但无法匹配'00A'，也就是说'00'后面只能是数字。 

> '\d\d\d'可以匹配'010'，只可匹配三位数字。  

> '\w\w\d'可以匹配'py3'，前两位可以是数字或者字母，但是第三位只能是数字。

> 'py.'可以匹配'pyc'、'pyo'、'py!'等。  

在正则表达式中，用*表示任意个字符（包括0个），用+表示至少一个字符，用?表示0个或1个字符，用{n}表示n个字符，用{n,m}表示n～m个字符。  


下面看一个复杂的例子：\d{3}\s+\d{3,8}。  
从左到右解读如下。  
（1）\d{3}表示匹配3个数字，如'010'。  
（2）\s可以匹配一个空格（也包括Tab等空白符），所以\s+表示至少有一个空格，如匹配' '，'   '等。  
（3）\d{3,8}表示匹配3～8个数字，如'1234567’。  

综合以上，上述正则表达式可以匹配以任意个空格隔开的带区号为3个数字、号码为3～8个数字的电话号码，如'021  8234567'。  


如果要匹配'010-12345'这样的号码呢？因为'-'是特殊字符，在正则表达式中，要用'\'转义，所以正则式是\d{3}\-\d{3,8}。  
但是，仍然无法匹配'010 - 12345'，因为'-'两侧带有空格，所以需要更复杂的匹配方式。要做更精确的匹配，可以用[ ]表示范围。  


```
[0-9a-zA-Z\_]可以匹配一个数字、字母或者下画线。  
[0-9a-zA-Z\_]+可以匹配至少由一个数字、字母或者下画线组成的字符串，如'a100'，'0_Z'，'Py3000'等。  
[a-zA-Z\_][0-9a-zA-Z\_]*可以匹配由字母或下画线开头，后接任意个由一个数字、字母或者下画线组成的字符串，也就是Python合法的变量。  
[a-zA-Z\_][0-9a-zA-Z\_]{0, 19}更精确地限制了变量的长度是1～20个字符（前面1个字符+后面最多19个字符）。  
A|B可以匹配A或B，所以(P|p)ython可以匹配'Python'或者'python'。  
^表示行的开头，^\d表示必须以数字开头。  
$表示行的结束，\d$表示必须以数字结束。  
需要注意，py也可以匹配'python'，但是加上^py$就变成了整行匹配，就只能匹配'py'了。  
```


<img src="image/Chapter5_1.png"  width="800" height="800">
<center>正则表达式常用符号</center>

# 5.2 re模块

对字符串'dog rat dog'调用re.match()函数，查找模式匹配'dog'。

In [5]:
import re
re.match(r'dog', 'dog rat dog')

<re.Match object; span=(0, 3), match='dog'>

In [6]:
m1  = re.match(r'dog', 'dog rat dog')
m1.group(0)

'dog'

但是，如果我们对 'rat'查找，则不会找到匹配。

In [8]:
re.match(r'rat', 'dog rat dog')

match()函数判断是否匹配，如果匹配成功，返回一个match对象，否则返回None。


In [9]:
test = '用户输入的字符串'
if re.match(r'正则表达式', test):
    print('ok')
else:
    print('failed')

failed


## 5.2.3 分组

除了简单地判断是否匹配之外，正则表达式还有提取子串的强大功能。用()表示的即为要提取的分组（Group）。


【例5-2】^(\d{3})-(\d{3,8})$分别定义了两个组，可以直接从匹配的字符串中提取出区号和本地号码。

In [10]:
m = re.match(r'^(\d{3})-(\d{3,8})$', '010-12345')
m

<re.Match object; span=(0, 9), match='010-12345'>

In [11]:
m.group(0)

'010-12345'

In [12]:
m.group(1)

'010'

In [13]:
m.group(2)

'12345'

如果正则表达式中定义了组，就可以在match对象上用group()方法提取出子串。注意到group(0)是原始字符串，group(1)、group(2)……表示第1、2……个子串。提取子串非常有用，举例如下。


In [15]:
t = '19:05:30'
m = re.match(r'^(0[0-9]|1[0-9]|2[0-3]|[0-9])\:(0[0-9]|1[0-9]|2[0-9]|3[0-9]|4[0-9]|5[0-9]|[0-9])\:(0[0-9]|1[0-9]|2[0-9]|3[0-9]|4[0-9]|5[0-9]|[0-9])$', t)

In [16]:
m.groups()

('19', '05', '30')

这个正则表达式可以直接识别合法的时间。但有些时候，用正则表达式也无法做到完全验证，识别日期代码如下。
> '^(0[1-9]|1[0-2]|[0-9])-(0[1-9]|1[0-9]|2[0-9]|3[0-1]|[0-9])$'

对于'2-30''4-31'这样的非法日期，用正则还是识别不了，或者说写出来非常困难，这时就需要程序配合识别了

# 5.3  贪婪匹配

需要特别指出的是正则匹配默认是贪婪匹配，也就是匹配尽可能多的字符。

【例5-3】匹配出数字后面的0。


In [17]:
re.match(r'^(\d+)(0*)$', '102300').groups()

('102300', '')

由于\d+采用贪婪匹配，直接把后面的0全部匹配了，结果0*只能匹配空字符串了。  
必须让\d+采用非贪婪匹配（也就是尽可能少匹配），才能把后面的0匹配出来，加个?就可以让\d+采用非贪婪匹配。


In [18]:
re.match(r'^(\d+?)(0*)$', '102300').groups()

('1023', '00')

# 5.4 编译

当我们在Python中使用正则表达式时，re模块内部会做两件事情。  
（1）编译正则表达式，如果正则表达式的字符串本身不合法，会报错。  
（2）用编译后的正则表达式去匹配字符串。  
如果一个正则表达式要重复使用几千次，出于效率的考虑，我们可以预编译该正则表达式，接下来重复使用时就不再需要编译这个步骤，直接匹配。  

In [19]:
import re
re_telephone = re.compile(r'^(\d{3})-(\d{3,8})$') # 编译
re_telephone.match('010-12345').groups()   # 使用

('010', '12345')

In [20]:
re_telephone.match('010-8086').groups()   # 使用

('010', '8086')

编译后生成Regular Expression对象，由于该对象自己包含了正则表达式，所以调用对应的方法时不用给出正则字符串。

# 5.5 正则函数

In [ ]:
在Python中，re模块提供了以下几个函数对输入的字符串进行准确的查询，具体如下。

* re.compile()
* re.match()
* re.search()
* re.findall()

每一个函数都接收一个正则表达式和一个待匹配的字符串

## 5.5.1 re.compile()函数

re.compile()函数编译正则表达式模式，返回一个对象。可以把常用的正则表达式编译成正则表达式对象，方便后续调用及提高效率。  
re.compile(pattern, flags=0)  
pattern：指定编译时的表达式字符串。  
flags：编译标志位，用来修改正则表达式的匹配方式。支持re.L|re.M同时匹配flags标志位参数。  
●  re.I(re.IGNORECASE)：使匹配对大小写不敏感。  
●  re.L(re.LOCAL)：做本地化识别（Locale-aware）匹配。  
●  re.M(re.MULTILINE)：多行匹配，影响^和 $。  
●  re.S(re.DOTALL)：使.匹配包括换行在内的所有字符。  
●  re.U(re.UNICODE)：根据Unicode字符集解析字符。这个标志影响\w，\W，\b，\B.。  
●  re.X(re.VERBOSE)：该标志通过给予更灵活的格式以便将正则表达式写得更易于理解。  


re.compile()函数的用法示例如下。

In [23]:
import re
content = 'Citizen wang, always fall in love with neighbour，WANG'
rr = re.compile(r'wan\w', re.I) # 不区分大小写
print(type(rr))

<class 're.Pattern'>


In [24]:
a = rr.findall(content)
print(type(a))
print(a)


<class 'list'>
['wang', 'WANG']


## 5.5.2 re.match()函数

re.match()函数总是从字符串“开头匹配”，并返回匹配的字符串的match对象<class '_sre.SRE_Match’>。
格式如下:
> re.match(pattern, string[, flags=0])  
> pattern：匹配模式，由re.compile获得。  
> string： 需要匹配的字符串。  

re.match()函数的工作方式是只有当被搜索字符串的开头匹配模式的时候，它才能查找到匹配对象。


【例5-4】对字符串'dog rat dog'调用re.match()函数，查找模式匹配'dog'。

In [26]:
import re
re.match(r'dog', 'dog rat dog')

<re.Match object; span=(0, 3), match='dog'>

In [27]:
m1  = re.match(r'dog', 'dog rat dog')
m1.group(0)

'dog'

但是，如果我们对 'rat'查找，则不会找到匹配。

In [28]:
re.match(r'rat', 'dog rat dog')

再如（见书P100）

## 5.5.3 re.search()函数

re.search()函数对整个字符串进行搜索匹配，返回第一个匹配的字符串的match对象。格式如下。  
re.search(pattern, string[, flags=0])  
pattern：匹配模式，由re.compile获得。  
string：需要匹配的字符串。  
re.search()函数和re.match()函数类似，不过re.search()函数不会限制我们只从字符串的开头查找匹配，在【例5-4】的字符串中查找'rat'会查找到一个匹配。


In [29]:
m21 = re.search(r'rat', 'dog rat dog')
m21.group(0)

'rat'

然而re.search()函数会在它查找到一个匹配项之后停止继续查找，因此在示例字符串中用re.search()函数查找'dog'只找到其首次出现的位置。


In [30]:
m22 = re.search(r'dog', 'dog rat dog')
m22.group(0)

'dog'

其他代码见P101

## 5.5.4 re.findall() 函数

In [ ]:
在Python中使用最多的查找方法是调用re.findall()函数。当我们调用re.findall()函数时可以非常简单地得到一个所有匹配模式的列表，而不是得到match的对象。对示例字符串调用re.findall()函数得到结果如下。

In [31]:
re.findall(r'dog', 'dog rat dog')

['dog', 'dog']

In [32]:
re.findall(r'rat', 'dog rat dog')

['rat']

## 5.5.5 字符串的替换和修改

re模块还提供了对字符串的替换和修改函数，它们比字符串对象提供的函数功能要强大。
> sub (rule , replace , target [,count] )  
> subn(rule , replace , target [,count] )  

在目标字符串中按规则查找匹配的字符串，再把它们替换成指定的字符串。我们可以指定被替换的次数，否则将替换所有匹配到的字符串。
第一个参数是正则规则，第二个参数是将要被替换的字符串，第三个参数是目标字符串，第四个参数是被替换的次数。这两个函数的唯一区别是返回值。sub返回一个被替换的字符串，subn返回一个元组，第一个元素是被替换的字符串，第二个元素是一个数字，表明产生了多少次替换。


【例5-5】将下面字符串中的'dog'全部替换成'cat'。

In [33]:
s='I have a dog , you have a dog , he have a dog'
re.sub( r'dog' , 'cat' , s )

'I have a cat , you have a cat , he have a cat'

如果只想替换前面两个，则可以写如下代码。

In [34]:
re.sub( r'dog' , 'cat' , s , 2 )

'I have a cat , you have a cat , he have a dog'

或者我们想知道发生了多少次替换，则可以使用subn。

In [35]:
re.subn( r'dog' , 'cat' , s )

('I have a cat , you have a cat , he have a cat', 3)

# 5.6 格式化输出

Python的格式化输出有两种方式：%和format。
format的功能要比%方式强大，其中format可以自定义字符填充空白、字符串居中显示、转换二进制、整数自动分割、百分比显示等功能。   
Python3.6之后新增了f格式化。

## 5.6.1 使用%进行格式化

首先看一个用%进行格式化的代码示例。

In [1]:
name1 = "Tomcat"
print("He said his name is %s." %name1)


He said his name is Tomcat.


字符串引号内的%为格式化的开始，类似于占位符，其后s表示占位处要补充的是字符型，紧跟在引号之后的%为需要填充的内容。使用这种方式进行字符串格式化时，要求被格式化的内容和格式字符之间必须一一对应。  
用%进行格式化时：%d表示数值（取整）占位，%f表示浮点型占位。


In [1]:
name1="Tomcat"
print("He said his name is %d."%name1)


TypeError: %d format: a real number is required, not str

In [2]:
"i am %(name)s age %(age)d" % {"name": "alex", "age": 18}

'i am alex age 18'

In [3]:
"percent %.2f" % 99.97623

'percent 99.98'

In [4]:
"i am %(pp).2f" % {"pp": 123.425556 }

'i am 123.43'

In [5]:
"i am %(pp)+.2f %%" % {"pp": 123.425556,} #两个%%表示输出一个%

'i am +123.43 %'

## 5.6.2 使用format()函数进行格式化

除了%字符串格式化方法之外，推荐使用format()函数进行格式化，该方法非常灵活，不仅可以使用位置进行格式化，还支持使用关键参数进行格式化。
Python中format()函数用于字符串的格式化。


### 1.通过关键字

In [6]:
print('{名字}今天{动作}'.format(名字='陈某某',动作='拍视频'))#通过关键字
grade = {'name' : '陈某某', 'fenshu': '59'}
print('{name}电工考了{fenshu}'.format(**grade))#通过关键字，可用字典当关键字传入值时，在字典前加**即可

陈某某今天拍视频
陈某某电工考了59


### 2.通过位置

In [7]:
print('{1}今天{0}'.format('拍视频','陈某某'))#通过位置
print('{0}今天{1}'.format('陈某某','拍视频'))

陈某某今天拍视频
陈某某今天拍视频


^、<、>分别表示居中、左对齐、右对齐，后面带宽度。

In [8]:
print('{:^14}'.format('陈某某'))  #共占位14个宽度，陈某某居中
print('{:>14}'.format('陈某某'))  #共占位14个宽度，陈某某居右对齐
print('{:<14}'.format('陈某某'))  #共占位14个宽度，陈某某居左对齐
print('{:*<14}'.format('陈某某')) #共占位14个宽度，陈某某居左对齐其他的*填充
print('{:&>14}'.format('陈某某')) #共占位14个宽度，陈某某居右对齐其他的&填充
#填充和对齐^<>分别表示居中、左对齐、右对齐，后面14是总宽度（一个汉字为一个宽度）


     陈某某      
           陈某某
陈某某           
陈某某***********
&&&&&&&&&&&陈某某


### 3.f方法格式化

在普通字符串前添加f或F前缀，其效果类似于%方式或者format()。
示例如下。

In [9]:
name1 = "Fred"
print("He said his name is %s." %name1)

He said his name is Fred.


In [10]:
print("He said his name is {name1}.".format(**locals()))

He said his name is Fred.


In [11]:
f"He said his name is {name1}."      #py3.6之后才有的新功能

'He said his name is Fred.'

# 5.7 实战体验：验证信息的正则表达式

在填写个人信息时，对有些信息需要进行验证，如手机号码、身份证、E-mail等。  
下面对从键盘输入的E-mail进行验证，代码如下。

In [12]:
import re  
text = input("Please input your E-mail：\n")  
if re.match(r'^\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*$', text):  
    print('E-mail is Right!')  
else:  
    print('Wrong!Please reset your right E-mail!')  


Please input your E-mail：
22@qq.com
E-mail is Right!


In [15]:
text = input("Please input your E-mail：\n")  
if re.match(r'^\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*$',text):  
   print('E-mail is Right!')  
else:  
   print('Wrong!Please reset your right E-mail!')  

Please input your E-mail：
123@
Wrong!Please reset your right E-mail!


对于判断输入的身份证号码，可以将匹配规则进行以下替换。  
> ^([0-9]){7,18}(x|X)?$   

或  

> ^\d{8,18}|[0-9x]{8,18}|[0-9X]{8,18}?$  

若是判断输入的是手机号码，则进行如下替换。  
>  ^(13[0-9]|14[5|7]|15[0|1|2|3|5|6|7|8|9]|18[0|1|2|3|5|6|7|8|9])\d{8}$  


下面收集和整理了一些可以判断的规则。
### 1．校验数字的表达式

```
（1）数字：^[0-9]*$。    
（2）n位的数字：^\d{n}$。    
（3）至少n位的数字：^\d{n,}$。    
（4）m～n位的数字：^\d{m,n}$。    
（5）零和非零开头的数字：^(0|[1-9][0-9]*)$。    
（6）非零开头的最多带两位小数的数字：^([1-9][0-9]*)+(.[0-9]{1,2})?$。    
（7）带1～2位小数的正数或负数：^(\-)?\d+(\.\d{1,2})?$。    
（8）正数、负数和小数：^(\-|\+)?\d+(\.\d+)?$。    
（9）有两位小数的正实数：^[0-9]+(.[0-9]{2})?$。    
（10）有1～3位小数的正实数：^[0-9]+(.[0-9]{1,3})?$。    
（11）非零的正整数：^[1-9]\d*$、^([1-9][0-9]*){1,3}$ 或^\+?[1-9][0-9]*$。   
（12）非零的负整数：^\-[1-9][]0-9"*$ 或^-[1-9]\d*$。    
（13）非负整数：^\d+$ 或^[1-9]\d*|0$。    
（14）非正整数：^-[1-9]\d*|0$ 或^((-\d+)|(0+))$。    
（15）非负浮点数：^\d+(\.\d+)?$ 或^[1-9]\d*\.\d*|0\.\d*[1-9]\d*|0?\.0+|0$。    
```

### 2．校验字符的表达式


```
（1）汉字：^[\u4e00-\u9fa5]{0,}$。
（2）英文和数字：^[A-Za-z0-9]+$ 或^[A-Za-z0-9]{4,40}$。
（3）长度为3～20的所有字符：^.{3,20}$。
（4）由26个英文字母组成的字符串：^[A-Za-z]+$。
（5）由26个大写英文字母组成的字符串：^[A-Z]+$。
（6）由26个小写英文字母组成的字符串：^[a-z]+$。
（7）由数字和26个英文字母组成的字符串：^[A-Za-z0-9]+$。
（8）由数字、26个英文字母或者下画线组成的字符串：^\w+$ 或^\w{3,20}$。
（9）中文、英文、数字包括下画线：^[\u4E00-\u9FA5A-Za-z0-9_]+$。
（10）中文、英文、数字但不包括下画线等符号：^[\u4E00-\u9FA5A-Za-z0-9]+$ 或^[\u4E00-\u9FA5A-Za-z0-9]{2,20}$。
```

### 3．特殊需求表达式


```
（1）E-mail地址：^\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*$。
（2）域名：[a-zA-Z0-9][-a-zA-Z0-9]{0,62}(/.[a-zA-Z0-9][-a-zA-Z0-9]{0,62})+/.?。
（3）InternetURL：[a-zA-z]+://[^\s]* 或^http://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?$。
（4）手机号码：^(13[0-9]|14[5|7]|15[0|1|2|3|5|6|7|8|9]|18[0|1|2|3|5|6|7|8|9])\d{8}$。
（5）电话号码（"×××-×××××××""××××-××××××××""×××-×××××××""×××-××××××××"、"×××××××"和"××××××××）：^(\(\d{3,4}-)|\d{3.4}-)? \d{7,8}$。
（6）国内电话号码（0511-4405222、021-87888822）：\d{3}-\d{8}|\d{4}-\d{7}。
（7）身份证号（15位、18位数字）：^\d{15}|\d{18}$。
（8）短身份证号码（数字、字母x结尾）：^([0-9]){7,18}(x|X)?$ 或^\d{8,18}|[0-9x]{8,18}| [0-9X]{8,18}?$。
（9）账号是否合法（字母开头，允许5～16字节，允许字母数字下画线）：^[a-zA-Z] [a-zA-Z0-9_]{4,15}$。
（10）密码（以字母开头，长度为6～18，只能包含字母、数字和下画线）：^[a-zA-Z]\w {5,17}$。
（11）强密码（必须包含大小写字母和数字的组合，不能使用特殊字符，长度为8～10）：^(?=.*\d)(?=.*[a-z])(?=.*[A-Z]).{8,10}$。
（12）日期格式：^\d{4}-\d{1,2}-\d{1,2}。
（13）一年的12个月（01～09和1～12）：^(0?[1-9]|1[0-2])$。
（14）一个月的31天（01～09和1～31）：^((0?[1-9])|((1|2)[0-9])|30|31)$。
（15）钱的输入格式，有4种钱的表示形式可以接收，"10000.00" 和 "10,000.00"，没有“分”的"10000"和"10,000"：^[1-9][0-9]*$。
（16）空白行的正则表达式：\n\s*\r (可以用来删除空白行)。
（17）首尾空白字符的正则表达式：^\s*|\s*$或(^\s*)|(\s*$)，可以用来删除行首行尾的空白字符（包括空格、制表符、换页符等）。
（18）中国邮政编码（6位数字）：[1-9]\d{5}(?!\d)。
（19）IP地址：\d+\.\d+\.\d+\.\d+ （提取IP地址时有用）。
（20）IP地址：((?:(?:25[0-5]|2[0-4]\\d|[01]?\\d?\\d)\\.){3}(?:25[0-5]|2[0-4]\\d|[01]?\\d?\\d))。
```